In [1]:
import torch
import itertools
import os
import sys
import torchvision.transforms as T
import torchvision.datasets as datasets
from PIL import Image
from tqdm import tqdm
from transformers import ViTForImageClassification
from torchvision.models import vision_transformer

from zennit.image import imgify
from zennit.composites import LayerMapComposite
import zennit.rules as z_rules

module_path = os.path.join(os.getcwd(), '.')
sys.path.append(module_path)

from lxt.efficient import monkey_patch, monkey_patch_zennit

# Modify the Vision Transformer module to compute Layer-wise Relevance Propagation (LRP)
# in the backward pass. For ViTs, we utilize the LRP Gamma rule. It is implemented
# inside the 'zennit' library. To make it compatible with LXT, we also monkey patch it. That's it.
monkey_patch(ViTForImageClassification, verbose=True)
monkey_patch_zennit(verbose=True)


def get_vit_imagenet(device="cuda"):
    """
    Load a pre-trained Vision Transformer (ViT) model with ImageNet weights.
    
    Parameters:
    device (str): Device to load the model on ('cuda' or 'cpu')
    
    Returns:
    tuple: (model, weights) - The ViT model and its pre-trained weights
    """
    weights =vision_transformer.ViT_B_16_Weights.IMAGENET1K_V1
    model = vision_transformer.vit_b_16(weights=weights)
    model.eval()
    model.to(device)
    
    # Deactivate gradients on parameters to save memory
    # for param in model.parameters():
    #     param.requires_grad = False
        
    return model, weights

# Load a couple of dog and cat images from ImageNet (torchvision)
def get_imagenet_samples(num_samples=2):
    transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    # Download ImageNet val subset (tiny-imagenet or use CIFAR10 for demo)
    # TODO: Mybe we should use a Hi Res dataset instead for the examples
    dataset = datasets.CIFAR10(root='../../src/experiments/data', train=False, download=True, transform=transform)
    # Class 3 = cat, Class 5 = dog
    imgs_list = []
    labels_list = []
    for image, label in dataset:
        imgs_list.append(image)
        labels_list.append(label)
        if len(imgs_list) >= num_samples:
            break
    return imgs_list, labels_list


C:\Users\Kevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Patched GELU
Patched LayerNorm
Patched MultiheadAttention
Patched Zennit BasicHook's forward
Patched Zennit BasicHook's backward


In [2]:
# Load the pre-trained ViT model
# model, weights = get_vit_imagenet()
model = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10')
model.to("cuda")
for param in model.parameters():
    param.requires_grad = False

imgs_list, labels_list = get_imagenet_samples(1000)

# Store the generated heatmaps
heatmaps = []



In [3]:
# Experiment with different gamma values for Conv2d and Linear layers
# Gamma is a hyperparameter in LRP that controls how much positive vs. negative
# contributions are considered in the explanation
# for conv_gamma, lin_gamma in itertools.product([0.1, 0.25, 100], [0, 0.01, 0.05, 0.1, 1]):
conv_gamma = 100
lin_gamma = 1
for i in tqdm(range(len(imgs_list))):
    input_tensor = imgs_list[i].unsqueeze(0).to("cuda")
    # input_tensor = weights.transforms()(image).unsqueeze(0).to("cuda")
    input_tensor.grad = None  # Reset gradients
    # print("Gamma Conv2d:", conv_gamma, "Gamma Linear:", lin_gamma)

    # Define rules for the Conv2d and Linear layers using 'zennit'
    # LayerMapComposite maps specific layer types to specific LRP rule implementations
    zennit_comp = LayerMapComposite([
        (torch.nn.Conv2d, z_rules.Gamma(conv_gamma)),
        (torch.nn.Linear, z_rules.Gamma(lin_gamma)),
    ])

    # Register the composite rules with the model
    zennit_comp.register(model)

    # Forward pass with gradient tracking enabled
    y = model(input_tensor.requires_grad_()).logits

    # Get the top 5 predictions
    _, top5_classes = torch.topk(y, 5, dim=1)
    top5_classes = top5_classes.squeeze(0).tolist()

    # Get the class labels
    labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
    # labels = weights.meta["categories"]
    top5_labels = [labels[class_idx] for class_idx in top5_classes]

    # Print the top 5 predictions and their labels
    # for i, class_idx in enumerate(top5_classes):
    #     print(f'Top {i+1} predicted class: {class_idx}, label: {top5_labels[i]}')

    # Backward pass for the highest probability class
    # This initiates the LRP computation through the network
    y[0, top5_classes[0]].backward()

    # Remove the registered composite to prevent interference in future iterations
    zennit_comp.remove()

    # Calculate the relevance by computing Gradient * Input
    # This is the final step of LRP to get the pixel-wise explanation
    heatmap = (input_tensor * input_tensor.grad).sum(1)

    # Normalize relevance between [-1, 1] for plotting
    heatmap = heatmap / abs(heatmap).max()

    # Store the normalized heatmap
    heatmaps.append(heatmap[0].detach().clone().cpu())

# torch.save(torch.stack(heatmaps), "zennit_lrp_attrs.pt")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:46<00:00,  6.02it/s]


In [4]:
torch.save(torch.stack(heatmaps), "attnlrp_cifar10_attrs.pt")